In [1]:
!pip install textattack
!pip install tensorflow-text
!pip install datasets
!pip install --upgrade datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 415 kB 15.5 MB/s 
     |████████████████████████████████| 41.4 MB 1.6 MB/s 
     |████████████████████████████████| 3.6 MB 35.9 MB/s 
     |████████████████████████████████| 60 kB 4.9 MB/s 
     |████████████████████████████████| 101 kB 6.4 MB/s 
     |████████████████████████████████| 769 kB 35.9 MB/s 
     |████████████████████████████████| 290 kB 48.8 MB/s 
     |████████████████████████████████| 4.2 MB 35.5 MB/s 
     |████████████████████████████████| 401 kB 43.3 MB/s 
     |████████████████████████████████| 212 kB 64.4 MB/s 
     |████████████████████████████████| 140 kB 65.3 MB/s 
     |████████████████████████████████| 1.1 MB 47.7 MB/s 
     |████████████████████████████████| 56 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 53.0 MB/s 
     |████████████████████████████████| 6.6 MB 38.7 MB/s 
     |██████████████████████████████

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/263 Adversarial')

Mounted at /content/drive


In [3]:
import pandas as pd

sst_tr_dataset = pd.read_csv('sst_train_data.csv')
sst_val_dataset = pd.read_csv('sst_val_data.csv')
sst_test_dataset = pd.read_csv('sst_test_data.csv')

cola_tr_dataset = pd.read_csv('cola_train_data.csv')
cola_val_dataset = pd.read_csv('cola_val_data.csv')
cola_test_dataset = pd.read_csv('cola_test_data.csv')

In [4]:
sst_val_dataset.sample(5)

,sentences,labels
376,I ca n't say that I liked Homeboy ; it 'd be m...,0
514,"It 's so good that its relentless , polished w...",1
62,Makes even the claustrophobic on-board quarter...,1
330,Gives you the steady pulse of life in a beauti...,1
418,The most compelling Wiseman epic of recent yea...,1


In [5]:
cola_val_dataset.sample(5)

,sentences,labels
401,She kicked her,1
253,A programme about Euripides is on a Radio 4 to...,1
0,The moon glows in the darkness.,1
205,"He can't possibly do that, can he",1
97,He kicked herself,0


In [9]:
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification, AutoTokenizer, pipeline

bert_sst = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2")
bert_sst_tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2")

# bert_cola = TFAutoModelForSequenceClassification.from_pretrained("Abirate/bert_fine_tuned_cola")
# bert_cola_tokenizer = AutoTokenizer.from_pretrained('Abirate/bert_fine_tuned_cola')
bert_cola = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-CoLA")
bert_cola_tokenizer = AutoTokenizer.from_pretrained('textattack/bert-base-uncased-CoLA')

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
import textattack
import transformers
from textattack import Attack, AttackArgs
from textattack.attack_recipes import FasterGeneticAlgorithmJia2019

model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(bert_sst, bert_sst_tokenizer)

# Construct our four components for `Attack`
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.semantics import WordEmbeddingDistance

goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper)
constraints = [RepeatModification(),
  StopwordModification(),
  WordEmbeddingDistance(min_cos_sim=0.8)]

from textattack.transformations import WordSwapEmbedding
from textattack.search_methods import GreedyWordSwapWIR

transformation = WordSwapEmbedding(max_candidates=50)
search_method = GreedyWordSwapWIR(wir_method="delete")

# Construct the actual attack
attack = Attack(goal_function, constraints, transformation, search_method)

# attack = textattack.attack_recipes.genetic_algorithm_alzantot_2018.GeneticAlgorithmAlzantot2018.build(model_wrapper)

# input_text = "I really enjoyed the new movie that came out last month."
# label = 1 #Positive
# attack_result = attack.attack(input_text, label)

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:10<00:00, 45.7MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpkn7coldn.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [49]:
from textattack import Attacker
from textattack.datasets import Dataset

new_dataset = []
for sentence, label in zip(sst_val_dataset['sentences'], sst_val_dataset['labels']):
  new_dataset.append((sentence, int(label)))

dataset = Dataset(new_dataset)

attack_args = AttackArgs(
    num_examples=200,
)

attacker = Attacker(attack, dataset, attack_args)
attack_set = attacker.attack_dataset()

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 





  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 1/200 [02:20<7:44:45, 140.13s/it]

[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   0%|          | 1/200 [02:20<7:44:47, 140.14s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (100%)]] --> [[[FAILED]]]

It 's a lovely film with lovely performances by Buy and Accorsi .






[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   1%|          | 2/200 [02:45<4:33:42, 82.94s/it] 

[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   1%|          | 2/200 [02:45<4:33:43, 82.95s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[1 (90%)]] --> [[0 (83%)]]

[[No]] one goes unindicted here , which is probably for the best .

[[Nos]] one goes unindicted here , which is probably for the best .




...

--------------------------------------------- Result 198 ---------------------------------------------
[[1 (100%)]] --> [[[FAILED]]]

Ruzowitzky has taken this mothball-y stuff and made a rather sturdy , old-fashioned entertainment out of it .






[Succeeded / Failed / Skipped / Total] 136 / 44 / 18 / 198: 100%|█████████▉| 199/200 [3:50:31<01:09, 69.50s/it]

[Succeeded / Failed / Skipped / Total] 137 / 44 / 18 / 199: 100%|█████████▉| 199/200 [3:50:31<01:09, 69.51s/it]

--------------------------------------------- Result 199 ---------------------------------------------
[[1 (100%)]] --> [[0 (100%)]]

In [[spite]] of [[Good]] [[Housekeeping]] 's unsavory characters and WWF mentality , this [[white]] trash [[War]] of the [[Roses]] is a [[surprisingly]] [[engaging]] [[film]] .

In [[sadness]] of [[Buena]] [[Housework]] 's unsavory characters and WWF mentality , this [[blanca]] trash [[Warfare]] of the [[Pink]] is a [[appallingly]] [[participate]] [[kino]] .






[Succeeded / Failed / Skipped / Total] 137 / 44 / 18 / 199: 100%|██████████| 200/200 [3:51:00<00:00, 69.30s/it]

[Succeeded / Failed / Skipped / Total] 138 / 44 / 18 / 200: 100%|██████████| 200/200 [3:51:01<00:00, 69.31s/it]

--------------------------------------------- Result 200 ---------------------------------------------
[[1 (100%)]] --> [[0 (99%)]]

Collateral Damage finally delivers the goods for Schwarzenegger [[fans]] .

Collateral Damage finally delivers the goods for Schwarzenegger [[amateurs]] .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 138    |
| Number of failed attacks:     | 44     |
| Number of skipped attacks:    | 18     |
| Original accuracy:            | 91.0%  |
| Accuracy under attack:        | 22.0%  |
| Attack success rate:          | 75.82% |
| Average perturbed word %:     | 22.92% |
| Average num. words per input: | 18.07  |
| Avg num queries:              | 46.95  |
+-------------------------------+--------+


In [50]:
successful_attacks = []
labels = []
for result in attack_set:
  if type(result) == textattack.attack_results.successful_attack_result.SuccessfulAttackResult:
    label = int(result.goal_function_result_str()[0])
    successful_attacks.append(result.perturbed_text())
    labels.append(label)

adversarial_df = pd.DataFrame({'sentences':successful_attacks, 'labels':labels})
adversarial_df.to_csv("adversial_sst.csv", index=False)
adversarial_df.head()

,sentences,labels
0,"Nos one goes unindicted here , which is probab...",1
1,And if you 're not circa moved to sobs by a co...,1
2,Uses brusque mood and vision into humans chara...,1
3,"Half Submarine flick , Half Ghost Story , Enti...",0
4,"Nothing 's at stake , just a twisty double-cro...",1


In [10]:
import textattack
import transformers
from textattack import Attack, AttackArgs
# from textattack.attack_recipes import FasterGeneticAlgorithmJia2019

model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(bert_cola, bert_cola_tokenizer)

# Construct our four components for `Attack`
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.semantics import WordEmbeddingDistance

goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper)
constraints = [RepeatModification(),
  StopwordModification(),
  WordEmbeddingDistance(min_cos_sim=0.8)]

from textattack.transformations import WordSwapEmbedding
from textattack.search_methods import GreedyWordSwapWIR

transformation = WordSwapEmbedding(max_candidates=50)
search_method = GreedyWordSwapWIR(wir_method="delete")

# Construct the actual attack
attack = Attack(goal_function, constraints, transformation, search_method)

# attack = textattack.attack_recipes.genetic_algorithm_alzantot_2018.GeneticAlgorithmAlzantot2018.build(model_wrapper)

# input_text = "I really enjoyed the new movie that came out last month."
# label = 1 #Positive
# attack_result = attack.attack(input_text, label)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:10<00:00, 45.4MB/s]
textattack: Unzipping file /root/.cache/textattack/tmptciy8hzx.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [11]:
from textattack import Attacker
from textattack.datasets import Dataset

new_dataset = []
for sentence, label in zip(cola_val_dataset['sentences'], cola_val_dataset['labels']):
  new_dataset.append((sentence, int(label)))

dataset = Dataset(new_dataset)

attack_args = AttackArgs(
    num_examples=200,
)

attacker = Attacker(attack, dataset, attack_args)
attack_set = attacker.attack_dataset()

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



  0%|          | 1/200 [00:24<1:19:47, 24.06s/it]

--------------------------------------------- Result 1 ---------------------------------------------


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/200 [00:25<1:23:14, 25.10s/it]

[[1 (98%)]] --> [[0 (76%)]]

The [[moon]] [[glows]] in the [[darkness]].

The [[luna]] [[shines]] in the [[dark]].




...

--------------------------------------------- Result 198 ---------------------------------------------
[[1 (53%)]] --> [[[SKIPPED]]]

Bill's reading Shakespeare and Maureen's singing Schubert satisfies me




[Succeeded / Failed / Skipped / Total] 152 / 32 / 15 / 199: 100%|█████████▉| 199/200 [1:20:08<00:24, 24.16s/it]

--------------------------------------------- Result 199 ---------------------------------------------
[[0 (98%)]] --> [[1 (86%)]]

[[Myself]] shaved me.

[[Thyself]] shaved me.




[Succeeded / Failed / Skipped / Total] 153 / 32 / 15 / 200: 100%|██████████| 200/200 [1:20:26<00:00, 24.13s/it]

--------------------------------------------- Result 200 ---------------------------------------------
[[0 (68%)]] --> [[1 (96%)]]

[[No]] reading Shakespeare satisfied me

[[Not]] reading Shakespeare satisfied me



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 153    |
| Number of failed attacks:     | 32     |
| Number of skipped attacks:    | 15     |
| Original accuracy:            | 92.5%  |
| Accuracy under attack:        | 16.0%  |
| Attack success rate:          | 82.7%  |
| Average perturbed word %:     | 24.73% |
| Average num. words per input: | 6.16   |
| Avg num queries:              | 15.18  |
+-------------------------------+--------+


In [12]:
successful_attacks = []
labels = []
for result in attack_set:
  if type(result) == textattack.attack_results.successful_attack_result.SuccessfulAttackResult:
    label = int(result.goal_function_result_str()[0])
    successful_attacks.append(result.perturbed_text())
    labels.append(label)

adversarial_df = pd.DataFrame({'sentences':successful_attacks, 'labels':labels})
adversarial_df.to_csv("adversial_cola.csv", index=False)
adversarial_df.head()

,sentences,labels
0,The luna shines in the dark.,1
1,The luna shines.,1
2,I sang a song with Mari while you did so with ...,1
3,Quoi Mary did with Bill was sing a song.,1
4,She attempting to leave,1
